In [348]:
#interface graphique 
import tkinter as tk
from tkinter import ttk
import pprint
import random
import time
#import de notebook
import nbimporter
from ia import Ia_dumb
Ia_dumb()
import sys

running


<h2><b>GAME</b></h2>

In [349]:
#classe principale 
class Snake_game :
    def __init__(self, tinker, size = 500, play_btn = True, game_speed = 120):
        #initialisation des variables
        self.game_size = size
        self.pixel_size = 20
        self.tinker = tinker
        self.game_speed = game_speed
        self.score = 0
        
        #initialisation fenêtre tkinter
        self.tinker.title("Snake")
        self.tinker.resizable(False, False)
        self.tinker.tk.call("tk", "scaling", 4.0)
        
        #initialisation de la matrice / creation d'une ligne vide
        self.grid_pos = [[] for i in range(0, int(self.game_size / self.pixel_size))]
        self.create_grid()
        
        #booleen pour afficher le bouton play 
        if play_btn : 
            #bouton jouer
            self.play_button = tk.Button(self.tinker, text = "PLAY", command = self.play, width = 8, height = 2)
            self.play_button.pack()
            
    def create_canvas(self) :
        #canvas
        canvas = tk.Canvas(self.tinker, width = self.game_size, height = self.game_size, background = "black", highlightthickness = 0)
        return canvas
          
    def create_grid(self) :        
        #initialisation d'un dictionnaire
        pos_dict = {}             
        #remplir chaque ligne avec une liste de tuple de coordonnées possibles  
        for x in range(0, self.game_size + 1, self.pixel_size) :
            pos_dict[x] = []
            for y in range(0, self.game_size + 1, self.pixel_size) :
                pos_dict[x].append((x, y))

        #création de matrice      
        for x in list(pos_dict.keys())[:-1]:
            for i in range(len(pos_dict[x])-1):
                self.grid_pos[i].append([pos_dict[x][i],pos_dict[x+self.pixel_size][i+1]])
        
    
    def play(self) :
        print("Playing")
        #retirer le bouton play 
        self.play_button.destroy()
        
        #initialisation canvas
        self.canvas = self.create_canvas()
        
        #on vérifie si il y a plus de 1 élément fils sur la fenêtre
        if len(self.tinker.winfo_children()) == 1 :
            #initialisation de la pomme
            self.apple = Apple(self.tinker, self.game_size, False)
            self.apple.create_apple(self.canvas)
            
            #initialisation du snake
            self.snake = Snake(self.tinker, self.game_size, False, self.canvas, self.apple)
            self.snake.init_snake()
            self.canvas.pack()
                    
    
    def game_over(self, canvas):
        #supprimer tous les éléments de tkinter sur le canvas
        canvas.delete(tk.ALL)

        #créer un champs de texte au milieu de l'écran
        canvas.create_text(
            canvas.winfo_width() / 2,
            canvas.winfo_height() / 2,
            text=f"GAME OVER ! Vous avez remporté {self.score} point(s).",
            fill="#FFFFFF",
            font=("Helvetica", 20)
        )

In [350]:
#TODO set Apple 
class Apple (Snake_game):
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        self.init_pos = int((self.game_size/self.pixel_size)/2)

    def create_apple(self, canvas):
        #print("apple", canvas)
        #création de la pomme
        p = self.init_pos -1
        self.apple = canvas.create_rectangle(
                self.grid_pos[p][p][0][1],
                self.grid_pos[p][p][0][0],
                self.grid_pos[p][p][1][1],
                self.grid_pos[p][p][1][0],
                outline="#000000",
                fill="#ff8f26",
                tags=('apple')
        )
        
    def update_apple(self, canvas, apple_id, snake_positions) :
        x = random.randint(0, int((self.game_size/self.pixel_size)) - 1)
        y = random.randint(0, int((self.game_size/self.pixel_size)) - 1)
        if (x,y) in snake_positions :
            self.update_apple(canvas, apple_id, snake_positions)
            return
        #dessin du snake
        canvas.coords(
            apple_id,
            self.grid_pos[x][y][0][1],
            self.grid_pos[x][y][0][0],
            self.grid_pos[x][y][1][1],
            self.grid_pos[x][y][1][0],
        )

In [363]:
class Snake (Snake_game) :
    def __init__(self, tinker, game_size, play_btn, canvas, apple):
        super().__init__(tinker, game_size, False)
        #initialisation position snake
        self.commands = {
            "z" : "top",
            "q" : "left",
            "d" : "right",
            "s" : "bottom"
        }
        self.reverse_commands = {
            "top" : "bottom",
            "right" : "left",
            "left" : "right",
            "bottom" : "top",
        }
        self.direction = "right"
        self.commands_history = [self.direction]
        self.snake_head = (7, 5)
        self.snake_positions = [self.snake_head, (6,5), (5,5)]
        self.canvas = canvas
        self.apple = apple
        #écouter touche du clavier
        self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas, self.apple))
        
    #fonction d'écoute de touche    
    def set_key_event(self, event, canvas, apple) :
        #vérifier la touche 
        if event.char in self.commands : 
            self.commands_history.append(self.commands[event.char])
            #on vérifie si l'utilisateur n'appuie pas deux fois sur la même touche
            if (self.commands_history[-1] != self.direction) and (self.commands_history[-1] != self.reverse_commands[self.direction]) :
                    #définir la direction
                    self.direction = self.commands[event.char]
                    print(self.direction, "changed")
        
    def init_snake(self):
        #print("snake", canvas)
        for i, position in enumerate(self.snake_positions) :
            x = position[0] - 1 if position[0] != 0 else 0
            y = position[1] - 1 if position[1] != 0 else 0
            self.snake_part = self.canvas.create_rectangle(
                self.grid_pos[x][y][0][1],
                self.grid_pos[x][y][0][0],
                self.grid_pos[x][y][1][1],
                self.grid_pos[x][y][1][0],
                outline="#000000",
                fill="#1EC81C",
                tags=('snake')
            )
        self.canvas.after(self.game_speed, self.perform_actions)
            
    def add_snake_part(self, posx, posy):
        x = posx - 1 if posx != 0 else 0
        y = posy - 1 if posy != 0 else 0
        self.snake_part = self.canvas.create_rectangle(
                self.grid_pos[x][y][0][1],
                self.grid_pos[x][y][0][0],
                self.grid_pos[x][y][1][1],
                self.grid_pos[x][y][1][0],
                outline="#000000",
                fill="#1EC81C",
                tags=('snake')
            )
    
    def update_snake(self, snake_id, position) :
        #dessin du snake
        x = position[0] - 1 if position[0] != 0 else 0
        y = position[1] - 1 if position[1] != 0 else 0
        self.canvas.coords(
            snake_id,
            self.grid_pos[x][y][0][1],
            self.grid_pos[x][y][0][0],
            self.grid_pos[x][y][1][1],
            self.grid_pos[x][y][1][0],
        )
            
               
    def perform_actions(self):
        if self.is_touched():
            self.game_over(self.canvas)

        self.move()
        self.canvas.after(self.game_speed, self.perform_actions)
           
    def move(self) :
        print("moving")
        last_x = self.snake_positions[-1][0]
        last_y = self.snake_positions[-1][1]
        
        head_x_position, head_y_position = self.snake_positions[0]
        if self.direction == "left":
            new_head_position = (head_x_position - 1, head_y_position)
        elif self.direction == "right":
            new_head_position = (head_x_position + 1, head_y_position)
        elif self.direction == "bottom":
            new_head_position = (head_x_position, head_y_position + 1)
        elif self.direction == "top":
            new_head_position = (head_x_position, head_y_position - 1)
            
        self.snake_positions = [new_head_position] + self.snake_positions[:-1]
        for segment, position in zip(self.canvas.find_withtag("snake"), self.snake_positions):
            try:
                if position[0] < 0 and position[1] < 0:
                    self.game_over(self.canvas)
                else :
                    print(position[0], position[1])
                    self.update_snake(segment, position)
            except IndexError:
                print("GameOver pas dans le tableau")
                self.game_over(self.canvas)
            
        self.apple_id = self.canvas.find_withtag("apple")
        self.snake_ids = self.canvas.find_withtag("snake")
        if len(self.snake_ids) > 0 : 
            head_coords = self.canvas.coords(self.snake_ids[0])
            apple_coords = self.canvas.coords(self.apple_id)
            if head_coords == apple_coords :
                self.apple.update_apple(self.canvas, self.apple_id, self.snake_positions)
                self.snake_positions.append((last_x, last_y))
                self.add_snake_part(last_x, last_y)
                self.score += 1
        
    def is_touched(self):
        #les coordonnées de la tête
        x, y = self.snake_positions[0]
        #vérifier si le serpent touche les bords ou lui-même
        if (x, y) in self.snake_positions[1:] :
            print("GAMEOVER", x, y)
            return True

In [364]:
root = tk.Tk()
snake_game = Snake_game(root)
root.mainloop()

Playing
moving
8 5
7 5
6 5
moving
9 5
8 5
7 5
moving
10 5
9 5
8 5
moving
11 5
10 5
9 5
moving
12 5
11 5
10 5
moving
13 5
12 5
11 5
bottom changed
moving
13 6
13 5
12 5
moving
13 7
13 6
13 5
moving
13 8
13 7
13 6
left changed
moving
12 8
13 8
13 7
moving
11 8
12 8
13 8
top changed
moving
11 7
11 8
12 8
moving
11 6
11 7
11 8
right changed
moving
12 6
11 6
11 7
moving
13 6
12 6
11 6
moving
14 6
13 6
12 6
moving
15 6
14 6
13 6
moving
16 6
15 6
14 6
bottom changed
moving
16 7
16 6
15 6
moving
16 8
16 7
16 6
moving
16 9
16 8
16 7
moving
16 10
16 9
16 8
moving
16 11
16 10
16 9
moving
16 12
16 11
16 10
moving
16 13
16 12
16 11
moving
16 14
16 13
16 12
moving
16 15
16 14
16 13
moving
16 16
16 15
16 14
left changed
moving
15 16
16 16
16 15
moving
14 16
15 16
16 16
moving
13 16
14 16
15 16
moving
12 16
13 16
14 16
moving
11 16
12 16
13 16
moving
10 16
11 16
12 16
moving
9 16
10 16
11 16
moving
8 16
9 16
10 16
moving
7 16
8 16
9 16
moving
6 16
7 16
8 16
moving
5 16
6 16
7 16
moving
4 16
5 16
6 16
